In [69]:
import gzip
import json
import re
import os
import sys
import pandas as pd
import numpy as np

In [70]:
df = pd.read_csv('/Users/abhi/Downloads/goodreads_with_spoilers.csv')

In [71]:
df.columns

Index(['Unnamed: 0', 'index', 'user_id', 'timestamp', 'review_sentences',
       'rating', 'has_spoiler', 'book_id', 'review_id'],
      dtype='object')

In [72]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89627 entries, 0 to 89626
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0        89627 non-null  int64 
 1   index             89627 non-null  int64 
 2   user_id           89627 non-null  object
 3   timestamp         89627 non-null  object
 4   review_sentences  89627 non-null  object
 5   rating            89627 non-null  int64 
 6   has_spoiler       89627 non-null  bool  
 7   book_id           89627 non-null  int64 
 8   review_id         89627 non-null  object
dtypes: bool(1), int64(4), object(4)
memory usage: 5.6+ MB


In [73]:
df=df.drop('index',axis=1)
df=df.drop('Unnamed: 0',axis=1)
df.head()

,user_id,timestamp,review_sentences,rating,has_spoiler,book_id,review_id
0,8842281e1d1347389f2ab93d60773d4d,2017-08-30,"[[0, 'This is a special book.'], [0, 'It start...",5,True,18245960,dfdbb7b0eb5a7e4c26d59a937e2e5feb
1,8842281e1d1347389f2ab93d60773d4d,2017-03-20,"[[0, 'A fun, fast paced science fiction thrill...",3,True,28684704,2ede853b14dc4583f96cf5d120af636f
2,8842281e1d1347389f2ab93d60773d4d,2016-04-25,"[[0, 'I really enjoyed this book, and there is...",4,True,25884323,332732725863131279a8e345b63ac33e
3,8842281e1d1347389f2ab93d60773d4d,2014-07-26,"[[0, 'What a fun series.'], [0, 'I loved Wool,...",4,True,17855756,9db60af739f79f40c0016b924a5326b4
4,8842281e1d1347389f2ab93d60773d4d,2014-07-26,"[[0, ""A fun fast paced book that sucks you in ...",4,True,13453029,46a6e1a14e8afc82d221fec0a2bd3dd0


In [74]:
df.shape

(89627, 7)

<b> Generate random user for recommendation from the dataset

In [76]:
#generating random choice my df
import random
def generate_user(df,bookcount):
    books=[]
    for i in range(bookcount):
        res = random.randint(1, df.shape[0])
        books.append(['1',(df['book_id'][res]),df['rating'][res]])
    mydf=pd.DataFrame(books,columns=['user_id','book_id','rating'])
    return mydf

In [77]:
mydf=generate_user(df,10)

In [78]:
mydf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  10 non-null     object
 1   book_id  10 non-null     int64 
 2   rating   10 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 368.0+ bytes


In [79]:
my_bookid= set(mydf['book_id'])

In [80]:
my_bookid

{345627,
 6068551,
 6408862,
 11418574,
 17284874,
 17406847,
 17447582,
 17567218,
 18007564,
 29939390}

<b> Select similiar users who read atleast one of the books of our preference

In [81]:
similiar_user_df = df[df['book_id'].isin(my_bookid)]

In [82]:
similiar_users = set(similiar_user_df['user_id'])

In [83]:
#Get a set of all the user id's who read one of the books in our preference
similiar_users

{'000883382802f2d95a3dd545bb953882',
 '00efe9bf7f110572763cac91479965c5',
 '017619f3daea3c2382652fc71cc3b3d7',
 '0295c82f29e4e5c42c6b927336cde14f',
 '02a765c7f623e37bb80f0fb230ef773d',
 '03820db43c04959f4400ca5ed17ff6e4',
 '04d333c960b4869fdc0eecdfb636be73',
 '059ad5a129b1908b15bef1489339e43c',
 '089ed94c60d8757a5c1f07ca65c612c0',
 '09f3678ca89828ab032bd20138ea3fcf',
 '0afe52984c9345dc732c1defdceff04f',
 '0bf0a99bf8709671d40dc7dec2e2077c',
 '0e4f890d19b4c26c00e71638cc281b65',
 '122c34d7c9834b67021a5cb59ae49b81',
 '192880a878c54aca2be34896229ed1cd',
 '192e99f58fccf1faba2f5b2d1eaa3d3f',
 '1b1eab8f06a7c48fea0165fc3c682383',
 '1bacd7aa1688c435c4edc17e24f14ebe',
 '1c9062def5d322838ac3503d499023cd',
 '1cc97d880b59bc3c6c51385b36293d37',
 '1e9698f39e8a69db0f93ffc7133c3fb6',
 '1f203434e203e159478b88c21e2761d5',
 '1fc58f53eb67bcf6c78af62087697449',
 '20668476e60d7719807f5fe4f4815a1e',
 '2128fb2da76ea3ef707abf79e43286ac',
 '219e7dbb0caa9825cba05ee7cd1ade5a',
 '22e0bcd7b09a09ad07094f1b6edb8fba',
 

In [84]:
similiar_users_df = df[df['user_id'].isin(similiar_users)]

In [85]:
similiar_users_df.shape

(6101, 7)

In [86]:
similiar_users_df.head()

,user_id,timestamp,review_sentences,rating,has_spoiler,book_id,review_id
493,3aee9278dddcc99d8e0af367dff3d037,2017-01-29,"[[0, ""I've read many reviews comparing this bo...",2,True,24480276,f296992380d872fb361d1173b26b94db
494,3aee9278dddcc99d8e0af367dff3d037,2016-03-06,"[[0, 'First, a warning: if you, like me, thoug...",2,True,23281690,aa814507bf207b9c6f50b54e366df7f9
495,3aee9278dddcc99d8e0af367dff3d037,2015-08-31,"[[0, 'This book is the third in a trilogy of c...",2,True,26033727,b68b4fdf83b184b82e1c78068cbe0a88
496,3aee9278dddcc99d8e0af367dff3d037,2015-06-01,"[[0, 'I loved Throne of Glass.'], [0, ""It's an...",5,True,22839894,ff02d5bbe7b4f18d48c8396c52cdec10
497,3aee9278dddcc99d8e0af367dff3d037,2014-03-25,"[[0, 'For more reviews from me, visit my blog,...",1,True,13449934,8550da26b02c56f77f6667e9b1385ac4


<b> Selecting the required attributes for collaborative filtering and also adding our target user to dataset to get simalirity

In [87]:
similiar_user_sub = pd.DataFrame(similiar_users_df,columns=['user_id','rating','book_id'])

In [88]:
#concat myuserdf with similiar user df
similiar_user_sub = pd.concat([mydf[["user_id", "book_id", "rating"]], similiar_user_sub])


In [89]:
similiar_user_sub 

,user_id,book_id,rating
0,1,17406847,1
1,1,6408862,5
2,1,17567218,3
3,1,6068551,5
4,1,17447582,5
...,...,...,...
89307,9bc14ec7ba585f3f812511f3a404d30b,17567218,4
89308,9bc14ec7ba585f3f812511f3a404d30b,81992,0
89309,9bc14ec7ba585f3f812511f3a404d30b,13642753,0
89310,9bc14ec7ba585f3f812511f3a404d30b,394424,4


In [90]:
similiar_user_sub["book_id"] = similiar_user_sub["book_id"].astype(str)
similiar_user_sub["user_id"] = similiar_user_sub["user_id"].astype(str)
similiar_user_sub["rating"] = pd.to_numeric(similiar_user_sub["rating"])

In [91]:
similiar_user_sub.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6111 entries, 0 to 89423
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  6111 non-null   object
 1   book_id  6111 non-null   object
 2   rating   6111 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 191.0+ KB


In [92]:
#Code to get unique index for each user and book
similiar_user_sub["user_index"] = similiar_user_sub["user_id"].astype("category").cat.codes
similiar_user_sub["book_index"] = similiar_user_sub["book_id"].astype("category").cat.codes

In [93]:
#No. of unique users
len(similiar_user_sub['user_index'].unique())

240

In [94]:
#No. of unique books
len(similiar_user_sub['book_index'].unique())

4152

<b> Creating a sparse matrix(as dense matrix is heavy for storage if large dataset found) for user id vs all bookid and filling it with ratings 

In [95]:

from scipy.sparse import coo_matrix

ratings_mat_coo = coo_matrix((similiar_user_sub["rating"], (similiar_user_sub["user_index"], similiar_user_sub["book_index"])))
ratings_mat = ratings_mat_coo.tocsr()


In [96]:
ratings_mat_coo.shape

(240, 4152)

In [97]:
similiar_user_sub[similiar_user_sub["user_id"]=='1']

,user_id,book_id,rating,user_index,book_index
0,1,17406847,1,13,1316
1,1,6408862,5,13,3502
2,1,17567218,3,13,1346
3,1,6068551,5,13,3431
4,1,17447582,5,13,1324
5,1,17284874,5,13,1243
6,1,29939390,3,13,3001
7,1,11418574,4,13,214
8,1,18007564,5,13,1482
9,1,345627,4,13,3211


In [98]:
myindex= similiar_user_sub[similiar_user_sub["user_id"]=='1']['user_index'].unique()[0]

In [99]:
myindex

13

<b> Retrieving the user_index for target user and calculating similarity score for each of the other users in our matrix

In [100]:
from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(ratings_mat[myindex,:], ratings_mat).flatten()

In [101]:
similarity

array([0.15803489, 0.04927603, 0.02017442, 0.13184275, 0.04081221,
       0.0560279 , 0.05714379, 0.19968077, 0.07963237, 0.11700909,
       0.03196014, 0.2665009 , 0.1478281 , 1.        , 0.14439775,
       0.06585792, 0.37688918, 0.37688918, 0.13997313, 0.11306675,
       0.05949966, 0.0181525 , 0.20684481, 0.02410591, 0.37688918,
       0.07623652, 0.07097234, 0.37688918, 0.19968077, 0.05311607,
       0.03164022, 0.30151134, 0.02482437, 0.12088726, 0.02412786,
       0.02070788, 0.37688918, 0.07425757, 0.11428758, 0.37688918,
       0.19334007, 0.37688918, 0.01478281, 0.04566425, 0.00983422,
       0.05998257, 0.09124148, 0.03143473, 0.20684481, 0.10303941,
       0.20320503, 0.13005122, 0.37688918, 0.10874545, 0.06639508,
       0.05977851, 0.04391728, 0.08128201, 0.13794015, 0.04570453,
       0.05161776, 0.37688918, 0.10184752, 0.03586711, 0.        ,
       0.01217674, 0.07208569, 0.08534859, 0.00785868, 0.37688918,
       0.37688918, 0.37688918, 0.06326118, 0.13430383, 0.06611

In [102]:
similarity.shape

(240,)

In [103]:
import numpy as np
#selecting indices of sorted similarity in descending order
indices = np.argpartition(similarity, -15)[-15:]

In [104]:
indices

array([ 52, 190,  69,  27,  41,  61, 201,  70,  71, 218,  24,  13, 128,
       182,  80])

In [105]:
#get final list of top 15
similar_users = similiar_user_sub[similiar_user_sub["user_index"].isin(indices)].copy()
similar_users 

,user_id,book_id,rating,user_index,book_index
0,1,17406847,1,13,1316
1,1,6408862,5,13,3502
2,1,17567218,3,13,1346
3,1,6068551,5,13,3431
4,1,17447582,5,13,1324
5,1,17284874,5,13,1243
6,1,29939390,3,13,3001
7,1,11418574,4,13,214
8,1,18007564,5,13,1482
9,1,345627,4,13,3211


In [106]:
type(list(my_bookid)[0])
#similar_users["book_id"].isin(list(my_bookid))

int

In [107]:
similar_users[similar_users["book_id"].isin(list(my_bookid))]

,user_id,book_id,rating,user_index,book_index


In [108]:
similar_users[similar_users["user_id"]=="1"]

,user_id,book_id,rating,user_index,book_index
0,1,17406847,1,13,1316
1,1,6408862,5,13,3502
2,1,17567218,3,13,1346
3,1,6068551,5,13,3431
4,1,17447582,5,13,1324
5,1,17284874,5,13,1243
6,1,29939390,3,13,3001
7,1,11418574,4,13,214
8,1,18007564,5,13,1482
9,1,345627,4,13,3211


In [109]:
similiar_users = similar_users[similar_users["user_id"]!="1"]


In [110]:
#Removing all the already read books if present 
similiar_users=similiar_users[~similiar_users['book_id'].isin(my_bookid)]
similiar_users

,user_id,book_id,rating,user_index,book_index
553,c67442bba7b5951fd7de2688aa7c35bc,18007564,3,190,1482
6042,3e053f61b720b5c68342a863b6a0a3fc,18007564,4,52,1482
10213,ba465cd96590354424809666cf242f3f,25324885,0,182,2560
10214,ba465cd96590354424809666cf242f3f,6408862,4,182,3502
13113,20668476e60d7719807f5fe4f4815a1e,18007564,5,24,1482
16109,49e969a64679fc5bc180b95891d29f01,18007564,2,71,1482
35054,4110899311319fccb0e646e03221ce99,18007564,1,61,1482
39325,d0379dc4420eb93516f7958f15afeb4f,6408862,5,201,3502
39385,79922bdf68f4e8d4274fd1207895691c,18007564,2,128,1482
50920,5158d3fc3f4a2189b7cb41c0e51c2a23,6408862,4,80,3502


<b> Calculating the mean and avg rating of all the books

In [45]:
book_recs = similar_users.groupby("book_id").rating.agg(['count', 'mean'])

In [46]:
our_book_list =list(book_recs.index)
our_book_list

['13477676',
 '155884',
 '17332968',
 '17403101',
 '17838475',
 '18079527',
 '20249',
 '27246122',
 '29056083',
 '9593911',
 '9917641']

# Now load relevant book data from goodreads_book.json

In [47]:
def load_data(file_name):
    count = 0
    data = []
    with gzip.open(file_name) as fin:
        for l in fin:
            d = json.loads(l)
            count += 1
            if(d['book_id'] in(our_book_list)):
                data.append(d)
            
            # break if reaches the 100th line
#             if (count > maxcount):
#                 break
    return data

In [48]:
book_rev_data = load_data('/Users/abhi/Downloads/goodreads_books.json.gz')

In [49]:
#Convert to dataframe and ectract useful columns
book_rev_data

[{'isbn': '1101475552',
  'text_reviews_count': '101',
  'series': ['176347', '599763'],
  'country_code': 'US',
  'language_code': 'eng',
  'popular_shelves': [{'count': '11579', 'name': 'to-read'},
   {'count': '2556', 'name': 'urban-fantasy'},
   {'count': '1315', 'name': 'fantasy'},
   {'count': '1143', 'name': 'paranormal'},
   {'count': '846', 'name': 'favorites'},
   {'count': '550', 'name': 'werewolves'},
   {'count': '544', 'name': 'currently-reading'},
   {'count': '494', 'name': 'romance'},
   {'count': '489', 'name': 'vampires'},
   {'count': '399', 'name': 'patricia-briggs'},
   {'count': '381', 'name': 'paranormal-romance'},
   {'count': '371', 'name': 'series'},
   {'count': '313', 'name': 'shifters'},
   {'count': '261', 'name': 'fae'},
   {'count': '252', 'name': 'fiction'},
   {'count': '248', 'name': 'magic'},
   {'count': '246', 'name': 'shapeshifters'},
   {'count': '242', 'name': 'mercy-thompson'},
   {'count': '241', 'name': 'supernatural'},
   {'count': '196', '

In [50]:
books_desc_df = pd.DataFrame(book_rev_data,columns=['book_id','title','url','average_rating','image_url'])

In [51]:
books_desc_df

,book_id,title,url,average_rating,image_url
0,9917641,"River Marked (Mercy Thompson, #6)",https://www.goodreads.com/book/show/9917641-ri...,4.26,https://images.gr-assets.com/books/1355081145m...
1,13477676,"Forgive Me, Leonard Peacock",https://www.goodreads.com/book/show/13477676-f...,3.93,https://images.gr-assets.com/books/1370282544m...
2,17332968,Reality Boy,https://www.goodreads.com/book/show/17332968-r...,3.85,https://images.gr-assets.com/books/1362502869m...
3,155884,"Pride and Prescience: Or, A Truth Universally ...",https://www.goodreads.com/book/show/155884.Pri...,3.52,https://s.gr-assets.com/assets/nophoto/book/11...
4,17403101,"Mid-Life Love (Mid-Life Love, #1)",https://www.goodreads.com/book/show/17403101-m...,4.03,https://images.gr-assets.com/books/1363830240m...
5,9593911,"Pandemonium (Delirium, #2)",https://www.goodreads.com/book/show/9593911-pa...,4.07,https://images.gr-assets.com/books/1310371414m...
6,20249,Hunters of Dune (Dune Chronicles #7),https://www.goodreads.com/book/show/20249.Hunt...,3.63,https://s.gr-assets.com/assets/nophoto/book/11...
7,18079527,The Impossible Knife of Memory,https://www.goodreads.com/book/show/18079527-t...,3.93,https://images.gr-assets.com/books/1405786728m...
8,27246122,"Red Queen (The Chronicles of Alice, #2)",https://www.goodreads.com/book/show/27246122-r...,3.81,https://images.gr-assets.com/books/1453056797m...
9,29056083,Harry Potter and the Cursed Child - Parts One ...,https://www.goodreads.com/book/show/29056083-h...,3.74,https://images.gr-assets.com/books/1470082995m...


In [52]:
book_recs = book_recs.merge(books_desc_df, how="inner", on="book_id")

In [53]:
book_recs

,book_id,count,mean,title,url,average_rating,image_url
0,13477676,4,4.500,"Forgive Me, Leonard Peacock",https://www.goodreads.com/book/show/13477676-f...,3.93,https://images.gr-assets.com/books/1370282544m...
1,155884,1,4.000,"Pride and Prescience: Or, A Truth Universally ...",https://www.goodreads.com/book/show/155884.Pri...,3.52,https://s.gr-assets.com/assets/nophoto/book/11...
2,17332968,1,3.000,Reality Boy,https://www.goodreads.com/book/show/17332968-r...,3.85,https://images.gr-assets.com/books/1362502869m...
3,17403101,1,2.000,"Mid-Life Love (Mid-Life Love, #1)",https://www.goodreads.com/book/show/17403101-m...,4.03,https://images.gr-assets.com/books/1363830240m...
4,17838475,1,3.000,"The Taking (The Taking, #1)",https://www.goodreads.com/book/show/17838475-t...,3.79,https://images.gr-assets.com/books/1381515013m...
5,18079527,1,4.000,The Impossible Knife of Memory,https://www.goodreads.com/book/show/18079527-t...,3.93,https://images.gr-assets.com/books/1405786728m...
6,20249,1,3.000,Hunters of Dune (Dune Chronicles #7),https://www.goodreads.com/book/show/20249.Hunt...,3.63,https://s.gr-assets.com/assets/nophoto/book/11...
7,27246122,1,3.000,"Red Queen (The Chronicles of Alice, #2)",https://www.goodreads.com/book/show/27246122-r...,3.81,https://images.gr-assets.com/books/1453056797m...
8,29056083,8,2.625,Harry Potter and the Cursed Child - Parts One ...,https://www.goodreads.com/book/show/29056083-h...,3.74,https://images.gr-assets.com/books/1470082995m...
9,9593911,5,3.800,"Pandemonium (Delirium, #2)",https://www.goodreads.com/book/show/9593911-pa...,4.07,https://images.gr-assets.com/books/1310371414m...


In [54]:
book_recs['average_rating']= book_recs['average_rating'].astype(float)
top_recs = book_recs[book_recs["average_rating"] >=3.5]
top_recs.rename({'count': 'review_count', 'mean': 'review_mean'}, axis=1, inplace=True)

In [55]:
def make_clickable(val):
    return '<a target="_blank" href="{}">Goodreads</a>'.format(val, val)

def show_image(val):
    return '<a href="{}"><img src="{}" width=50></img></a>'.format(val, val)
top_recs.style.format({'url': make_clickable, 'image_url': show_image})

,book_id,count,mean,title,url,average_rating,image_url
0,13477676,4,4.500000,"Forgive Me, Leonard Peacock",Goodreads,3.930000,
1,155884,1,4.000000,"Pride and Prescience: Or, A Truth Universally Acknowledged (Mr. & Mrs. Darcy Mysteries, #1)",Goodreads,3.520000,
2,17332968,1,3.000000,Reality Boy,Goodreads,3.850000,
3,17403101,1,2.000000,"Mid-Life Love (Mid-Life Love, #1)",Goodreads,4.030000,
4,17838475,1,3.000000,"The Taking (The Taking, #1)",Goodreads,3.790000,
5,18079527,1,4.000000,The Impossible Knife of Memory,Goodreads,3.930000,
6,20249,1,3.000000,Hunters of Dune (Dune Chronicles #7),Goodreads,3.630000,
7,27246122,1,3.000000,"Red Queen (The Chronicles of Alice, #2)",Goodreads,3.810000,
8,29056083,8,2.625000,"Harry Potter and the Cursed Child - Parts One and Two (Harry Potter, #8)",Goodreads,3.740000,
9,9593911,5,3.800000,"Pandemonium (Delirium, #2)",Goodreads,4.070000,


In [58]:
mydf

,user_id,book_id,rating
0,1,17838475,3
1,1,17403101,2
2,1,155884,4
3,1,9917641,4
4,1,20249,3
5,1,13477676,4
6,1,18079527,4
7,1,27246122,3
8,1,9593911,4
9,1,29056083,3


,book_id,rating_count,Recommendation_Score,title,url,average_rating,image_url
0,13477676,4,4.500,"Forgive Me, Leonard Peacock",https://www.goodreads.com/book/show/13477676-f...,3.93,https://images.gr-assets.com/books/1370282544m...
1,155884,1,4.000,"Pride and Prescience: Or, A Truth Universally ...",https://www.goodreads.com/book/show/155884.Pri...,3.52,https://s.gr-assets.com/assets/nophoto/book/11...
2,17332968,1,3.000,Reality Boy,https://www.goodreads.com/book/show/17332968-r...,3.85,https://images.gr-assets.com/books/1362502869m...
3,17403101,1,2.000,"Mid-Life Love (Mid-Life Love, #1)",https://www.goodreads.com/book/show/17403101-m...,4.03,https://images.gr-assets.com/books/1363830240m...
4,17838475,1,3.000,"The Taking (The Taking, #1)",https://www.goodreads.com/book/show/17838475-t...,3.79,https://images.gr-assets.com/books/1381515013m...
5,18079527,1,4.000,The Impossible Knife of Memory,https://www.goodreads.com/book/show/18079527-t...,3.93,https://images.gr-assets.com/books/1405786728m...
6,20249,1,3.000,Hunters of Dune (Dune Chronicles #7),https://www.goodreads.com/book/show/20249.Hunt...,3.63,https://s.gr-assets.com/assets/nophoto/book/11...
7,27246122,1,3.000,"Red Queen (The Chronicles of Alice, #2)",https://www.goodreads.com/book/show/27246122-r...,3.81,https://images.gr-assets.com/books/1453056797m...
8,29056083,8,2.625,Harry Potter and the Cursed Child - Parts One ...,https://www.goodreads.com/book/show/29056083-h...,3.74,https://images.gr-assets.com/books/1470082995m...
9,9593911,5,3.800,"Pandemonium (Delirium, #2)",https://www.goodreads.com/book/show/9593911-pa...,4.07,https://images.gr-assets.com/books/1310371414m...
